In [ ]:
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.ipac.ned import Ned
import numpy as np
from astropy.cosmology import WMAP9 as cosmo
from urllib.request import urlretrieve
from collections import Counter
import matplotlib.pyplot as plt
import pickle
from astropy.io import fits
from astropy.table import Table
import glob
import os
from scipy.interpolate import CubicSpline
import textwrap
import seaborn as sns


def make_desig(data, ra_key='ra', dec_key='dec'):
    """make designation if df has 'ra' and 'dec' columns"""
    desig=[]
    for posstring in SkyCoord(data[ra_key].values*u.deg, data[dec_key].values*u.deg).to_string("hmsdms"):
        posstring = posstring.split(' ')
        des_ra = posstring[0][0:2]+posstring[0][3:5]
        des_dec = posstring[1][0:3]+posstring[1][4:6]
        desig.append('J'+des_ra+des_dec)
    return desig

def pos(row):
    """make skyCoord object for HST coord cone search"""
    return SkyCoord(ra=row['RA']*u.deg, dec=row['DEC']*u.deg)

def cross_bigmac(mul171):
    """crossmatch sample with big MAC"""
    # read in big  mac
    bigmac = pd.read_csv("/home/insepien/research-data/GFG.csv")
    # format designation
    desigs = []
    for i in range(len(bigmac)):
        name = bigmac['Name1'].loc[i].replace("SDSS","")
        if name[0] == "J":
            if "+" in name:
                desig = name.split("+")[0][:5] + "+" + name.split("+")[1][:4]
                desigs.append(desig)
            elif "-" in name:
                desig = name.split("-")[0][:5] + "-" + name.split("-")[1][:4]
                desigs.append(desig)
            else: print(name) 
        else:
            desigs.append(name)
    bigmac['DESIG'] = desigs

    # merge big mac and mullaney
    mul_bm = pd.merge(mul171,bigmac, on="DESIG")

    # optionally can get decals images
    # for n in mul_bm.index:
    #     urlretrieve('http://legacysurvey.org/viewer/jpeg-cutout?ra='+str(mul_bm.loc[n,'RA'])+'&dec='+str(mul_bm.loc[n,'DEC'])+'&layer=decals-dr7&pixscale=0.27&bands=grz',
    #                 "/home/insepien/research-data/hst/mul_bm/"+str(mul_bm.loc[n,'DESIG'])+'.jpg')
    return mul_bm, bigmac

def cal_sep(theta, z):
    """return dual sep in kpc given scalar angle sep in arcsec"""
    angle = (theta*u.arcsec).to(u.rad).value
    return (cosmo.angular_diameter_distance(z)*angle).to(u.kpc)


def f(on, theta):
    """plot decal image and annulus at detected dual separation theta"""
    fn = "/home/insepien/research-data/hst/mul_bm/"+on+".jpg"
    decals_plate_scale = 0.236 #''/pix
    pix_sep = theta/decals_plate_scale

    fig,ax = plt.subplots()
    im = plt.imread(fn)
    midF = im.shape[0]/2
    ax.imshow(im)
    circ = plt.Circle((midF,midF), pix_sep, fill=False, color='white',alpha=0.5,label=f"{theta:.2f}''")
    ax.add_patch(circ)
    ax.legend()
    ax.set_title(on)


In [ ]:
cal_sep(3,0.2)

### find megellan seeing

In [ ]:
fn = os.listdir('/home/insepien/research-data/agn-result/box/final_cut')
onames = [f[:10] for f in fn]

fwhms = []
min_seps = []
for on in onames:
    expfile = glob.glob(os.path.expanduser("~/raw-data-agn/exp-fits-agn/*"+on+"*.exp.fits"))[0]
    with fits.open(os.path.expanduser(expfile)) as hdul:
        hdu0 = hdul[0]
    z = mul171[mul171['DESIG']==on]['Z'].values[0]
    fwhm = hdu0.header['FWHM_AVE']
    fwhms.append(fwhm)
    min_seps.append(cal_sep(fwhm,z).value)

fig,ax = plt.subplots(1,2,figsize=(7,3))
ax[0].hist(fwhms)
ax[0].set_xlabel("Avg FWHM ['']")
ax[0].set_title(f"Min: {np.min(fwhms):.2f}, \n median: {np.median(fwhms):.2f}")

ax[1].hist(min_seps)
ax[1].set_xlabel("min resolution (kpc)")
ax[1].set_title(f"Min: {np.min(min_seps):.2f}, \n median: {np.median(min_seps):.2f}")
;

## cross-match catalogs

In [ ]:
# read in original sample
mul171 = pd.read_pickle("/home/insepien/research-data/alpaka/mull152.pkl")
mul171['DESIG'] = make_desig(mul171, ra_key='RA',dec_key="DEC")
mul_bm, bigmac = cross_bigmac(mul171)

# get some stats on sub-kpc pairs for science justification
subkpc_mask = ((bigmac['Primary System Type']=='Dual AGN Candidate') | (bigmac['Primary System Type']=='Dual AGN')) & (bigmac['Sep(kpc)']<1) & (bigmac['Sep(kpc)']>0)
subkpc_dual = bigmac[subkpc_mask]
dual_mask = ((bigmac['Primary System Type']=='Dual AGN Candidate') | (bigmac['Primary System Type']=='Dual AGN')) & (bigmac['Sep(kpc)']>1)
dual = bigmac[dual_mask]
print("fraction of sub-kpc", len(subkpc_dual)/len(dual))
print("number of duals", len(dual))

# get some methods of measuring sub-kpc sep
anyl_meth = Counter(subkpc_dual['Parsed Analysis Method'])
print("1st most common method: ",anyl_meth.most_common(2)[0])
print("2nd most common method: ",anyl_meth.most_common(2)[1])


# cross-match specifically for magellan
with open("/home/insepien/research-data/alpaka/alpaka_39fits.pkl","rb") as f:
    magel = pickle.load(f)

magel.reset_index(inplace=True)
magel.rename(columns={"Desig":"DESIG"},inplace=True)
magel_bm, _ = cross_bigmac(magel)

In [ ]:
from matplotlib.lines import Line2D
import matplotlib.patches as patches
sns.set_context("paper",font_scale=1.75)
sns.set_style('ticks')
sns.set_palette('colorblind')
figparams = {'font.family': 'DejaVu Sans',
            'font.serif':'Times',
            'hatch.linewidth' : 3.0}
plt.rcParams.update(figparams)

fig,ax = plt.subplots(figsize=(15,7),dpi=500)
# plot hst resolution limit
for reso in [1,5,20]:
    seeing = 0.04*2.5*reso
    ax.plot(np.linspace(0,3),cal_sep(seeing, np.linspace(0,3)),c='k',linestyle="--",alpha=0.5)
    ax.text(2.5, 1*reso, f"{seeing:.1f} arcsec", color='k',fontsize=15)

# add survey vol
rect = patches.Rectangle((0.1, 0), 0.4-0.1, 1, linewidth=2, edgecolor='w', facecolor='darkseagreen',alpha=0.3)
ax.add_patch(rect)
ax.text(0.2,1,"Our survey",c="darkolivegreen",fontsize=15)

# kpc points ie sep>1 with confidence>0.5, z0-3, have been imaged
conf_kpc = dual['ST1 Confidence Flag']>=0.5
imag_kpc = dual['Parsed Analysis Method'].str.contains("Imaging")
z_mask = (dual['z1']>0)&(dual['z1']<3)
kpcdf = dual[conf_kpc&imag_kpc&z_mask]
paper = list(kpcdf['Paper(s)'].str.split(" ; ").explode().value_counts().keys())[:15]
all_markers = list(Line2D.markers.keys())[5:]
all_colors = sns.color_palette("colorblind", len(paper))
for j in range(len(paper)):    
    papermask = np.array([np.isin(paper[j],kpcdf.loc[i,'Paper(s)'].split(" ; ")).item() for i in kpcdf.index.values])
    for k in kpcdf[papermask].index.values:
        ulabel = paper[j] if k==kpcdf[papermask].index.values[0] else None
        plt.scatter(kpcdf[papermask]['z1'][k],kpcdf[papermask]['Sep(kpc)'][k],marker=all_markers[j],c=all_colors[j],s=30,label=ulabel)

# plot points with confidence >0.5 and have been imaged
confidence_mask = subkpc_dual['ST1 Confidence Flag']>=0.5
imaging_mask  = subkpc_dual['Parsed Analysis Method'].str.contains("Imaging")
distance_mask = subkpc_dual['Sep(kpc)']>0.2  # to remove radio sources
df = subkpc_dual[confidence_mask&imaging_mask&distance_mask]
lagn = np.log10([0.033e44,0.135e44,3e44,10**43.23*600,6e46])
subkpc_markers = list(Line2D.markers.keys())[1:6]
for i,m,mrk in zip(df.index.to_list(),lagn,subkpc_markers): # mark confirmed and very sure candidates differently
    wrapped_label = "\n".join(textwrap.wrap(df['Paper(s)'][i], width=60))
    sca = ax.scatter(df['z1'][i], df['Sep(kpc)'][i],label=wrapped_label,s=60,cmap='magma',c=m,marker=mrk,vmin=np.min(lagn)-0.5,vmax=np.max(lagn)+1)

cbar_ax = fig.add_axes([0.99, 0.125, 0.01, 0.75]) 
cbar = fig.colorbar(sca,cax=cbar_ax)
cbar.set_label("Log($L_{AGN}$) [erg/s]")

ax.set_xlabel("Redshift")
ax.set_ylabel("Projected separation [kpc]")
ax.set_xlim((-0.01,3))
ax.set_ylim((0.2,110))
# set top xlabel to look back time
ax_top = ax.secondary_xaxis("top")
ax_top.set_xlabel("Lookback time [Gyr]")
# interpolate to get tick positions for round lookback time
spl = CubicSpline(cosmo.lookback_time(np.linspace(0,0.5)),np.linspace(0,0.5))
xtick_pos = spl(np.arange(1,12,2))
ax_top.set_xticks(xtick_pos)
ax_top.set_xticklabels(np.arange(1,12,2))
ax.set_yscale('log')

ax.legend(ncol=3,fontsize=8,loc='lower center')
ax.grid(linestyle='--',alpha=0.5)
fig.tight_layout()
fig.savefig("hst.png",dpi=500);

## binary calculations

In [ ]:
from scipy.integrate import quad

def f(x,alpha,beta,phi):
    """integrand of SChecter function"""
    return phi*x**alpha*np.exp(-x**beta)  

def n_massive_dens(logMcut_,logMbhcut_):
    """calculate the number density of binary given mass cuts"""
    # GSMF constants
    phi1 = 10**-4.85
    phi2 = 10**-2.85
    Ms = 10**11.33
    alpha1 = 0.92
    alpha2 = -1.38
    # BHMF constants
    alpha_bh = -1.27
    beta_bh = 0.45
    phi_bh = 10**-2
    Msbh = 10**8.09
    # numerically integrate the number density of massive galaxies (note the GSMF is a sum of 2 PS functions)
    result1, _ = quad(f, 10**logMcut_/Ms, np.inf, args=(alpha1,1,phi1))
    result2, _ = quad(f, 10**logMcut_/Ms, np.inf, args=(alpha2,1,phi2)) 
    n_massive_gal = result1+result2
    # number density of massive BHs
    n_massive_bh, _ = quad(f, 10**logMbhcut_/Msbh, np.inf, args=(alpha_bh,beta_bh,phi_bh)) 
    return n_massive_gal, n_massive_bh

def n_bi(n_massive_,n_binary_):
    """calculate number of binary and galaxies given their number densities
        find dual fraction and expected observations"""
    # get binary number 
    # SDSS covers only part of sky
    f_sky = 9380/41253
    # comoving volume in z-range
    volume = (cosmo.comoving_volume(0.4)-cosmo.comoving_volume(0.1)).value
    # Number of binary
    N_c22 = n_binary_*volume*f_sky
    # get massive gal number
    N_massive_gal = n_massive_*volume*f_sky
    # dual fraction
    f_dual = N_c22/N_massive_gal
    # results
    print(f"{N_c22:.0f} binaries in SDSS in z=0.14-0.22")
    print(f"{N_massive_gal:.2e} massive galaxies")
    print(f"dual fraction:{f_dual*100:.2f}%")
    print(f"We can observe {f_dual*50:.0f}/50 binaries")

In [ ]:
# define cuts and binary density
logMcut = 11
logMbhcut = 8.32
n_bin = 2.3e-4
# calculate number of massive things
n_massive_gal, n_massive_bh = n_massive_dens(logMcut_=logMcut, logMbhcut_=logMbhcut)

# print results
print("GSMF")
n_bi(n_massive_gal,n_bin)
print("\nBHMF")
n_bi(n_massive_bh,n_bin)